In [2]:
import numpy as np
import pandas as pd
from sbfl.base import SBFL
from scipy.stats import rankdata

#   e1,e2,e3,e4,e5,e6
X = np.array([
    [1,1,0,0,1,1], # coverage of test t0
    [1,1,0,0,1,1], # coverage of test t1
    [0,1,1,0,1,0], # coverage of test t2
    [1,0,0,1,0,0],
    [0,1,0,1,1,0],
], dtype=bool)

y = np.array([1,1,0,0,0], dtype=bool)

X, y

(array([[ True,  True, False, False,  True,  True],
        [ True,  True, False, False,  True,  True],
        [False,  True,  True, False,  True, False],
        [ True, False, False,  True, False, False],
        [False,  True, False,  True,  True, False]]),
 array([ True,  True, False, False, False]))

In [3]:
sbfl = SBFL(formula='Ochiai')
sbfl.fit_predict(X, y)

array([0.33333333, 0.57735027, 0.57735027, 0.81649658, 0.57735027,
       0.        ])

In [4]:
sbfl.ranks(method='max')

array([5, 4, 4, 1, 4, 6])

In [5]:
names = ['file', 'method', 'line']
elements = [
    ('file1.py', 'method1', 'line1'),
    ('file2.py', 'method1', 'line1'),
    ('file2.py', 'method1', 'line2'),
    ('file2.py', 'method2', 'line3'),
    ('file2.py', 'method2', 'line4'),
    ('file2.py', 'method3', 'line5'),
]
score_df = sbfl.to_frame(elements=elements, names=names)
score_df

score
file     method  line           
file1.py method1 line1  0.333333
file2.py method1 line1  0.577350
                 line2  0.577350
         method2 line3  0.816497
                 line4  0.577350
         method3 line5  0.000000

In [6]:
from sbfl.score_aggregation import dense_rank_based_suspiciousness_aware_voting

dense_rank_based_suspiciousness_aware_voting(score_df, ['file', 'method'])

score
file     method           
file1.py method1  0.111111
file2.py method1  0.577350
         method2  1.105172
         method3  0.000000

# Read GCOV files

In [7]:
gcov_dir = {
    'test100': 'resources/yara-buggy#3-100',
    'test101': 'resources/yara-buggy#3-101',
    'test102': 'resources/yara-buggy#3-102',
    'test103': 'resources/yara-buggy#3-103',
    'test104': 'resources/yara-buggy#3-104',
}
gcov_files = {test:[] for test in gcov_dir}
for test in gcov_dir:
    for path in os.listdir(gcov_dir[test]):
        if path.endswith('.gcov'):
            gcov_files[test].append(os.path.join(gcov_dir[test], path))
    print(f"{test}: {len(gcov_files[test])} gcov files are found.")

test100: 48 gcov files are found.
test101: 48 gcov files are found.
test102: 48 gcov files are found.
test103: 48 gcov files are found.
test104: 48 gcov files are found.


In [8]:
from sbfl.utils import gcov_files_to_frame, get_sbfl_scores_from_frame

cov_df = gcov_files_to_frame(gcov_files, only_covered=True, verbose=True)
cov_df

100%|██████████| 5/5 [00:01<00:00,  3.67it/s]


test100  test101  test102  test103  test104
file        function line                                             
threading.c NaN      172        12       12       12       12       12
                     175        12       12       12       12       12
                     178        12       12       12       12       12
                     182         8        8        8        8        8
                     185         8        8        8        8        8
...                            ...      ...      ...      ...      ...
hex_lexer.l NaN      252         1        1        1        0        0
                     253         1        1        1        0        0
                     254         1        1        1        0        0
                     256         1        1        1        0        0
                     262         1        1        1        0        0

[778 rows x 5 columns]

In [9]:
score_df = get_sbfl_scores_from_frame(cov_df, failing_tests=['test102'])

In [10]:
dense_rank_based_suspiciousness_aware_voting(score_df, ['file'])

,score
file,
atoms.c,42.570357
bitmask.c,0.000000
hex_grammar.c,17.897858
hex_grammar.y,5.070765
hex_lexer.c,37.720218
hex_lexer.l,5.102112
libyara.c,4.360333
mem.c,1.006231
modules.c,1.118034
